In [215]:
%pip install opencv-python
%pip install numpy
import numpy as np
import cv2 as cv
import copy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Question 2 compression avec LZW

In [216]:
# Get data in this cell
message = "ABAABAABACABBABCDAADACABABAAABAABBABABAABAAB"

In [217]:
def encode(message : str):
    dictsymb =[message[0]]
    dictbin = ["{:b}".format(0)]
    nbsymboles = 1
    for i in range(1,len(message)):
        if message[i] not in dictsymb:
            dictsymb += [message[i]]
            dictbin += ["{:b}".format(nbsymboles)] 
            nbsymboles +=1    
    longueurOriginale = np.ceil(np.log2(nbsymboles))*len(message)   
    for i in range(nbsymboles):
        dictbin[i] = "{:b}".format(i).zfill(int(np.ceil(np.log2(nbsymboles))))  
        dictsymb.sort()
        dictionnaire = np.transpose([dictsymb,dictbin])
    basicdictsymb = copy.deepcopy(dictsymb)
    basicdictbin = copy.deepcopy(dictbin)
        
    #encode
    i=0
    messageCode = []
    longueur = 0
    while i < len(message):
        precsouschaine = message[i] #sous-chaine qui sera codé
        souschaine = message[i] #sous-chaine qui sera codé + 1 caractère (pour le dictionnaire)

        #Cherche la plus grande sous-chaine. On ajoute un caractère au fur et à mesure.
        while souschaine in dictsymb and i < len(message):
            i += 1
            precsouschaine = souschaine
            if i < len(message):  #Si on a pas atteint la fin du message
                souschaine += message[i]  

        #Codage de la plus grande sous-chaine à l'aide du dictionnaire  
        codebinaire = [dictbin[dictsymb.index(precsouschaine)]]
        messageCode += codebinaire
        longueur += len(codebinaire[0]) 
        #Ajout de la sous-chaine codé + symbole suivant dans le dictionnaire.
        if i < len(message):
            dictsymb += [souschaine]
            dictbin += ["{:b}".format(nbsymboles)] 
            nbsymboles +=1

        #Ajout de 1 bit si requis
        if np.ceil(np.log2(nbsymboles)) > len(messageCode[-1]):
            for j in range(nbsymboles):
                dictbin[j] = "{:b}".format(j).zfill(int(np.ceil(np.log2(nbsymboles))))
    return messageCode, basicdictsymb, basicdictbin

In [218]:
def decode(message : str, basicdictsymb, basicdictbin):    
    dictsymb = basicdictsymb
    dictbin = basicdictbin
    
    # decode the message
    i=0
    decodedMessage = ""
    offset = 0
    nbsymboles = len(basicdictsymb)
    nbBitUsed = int(np.ceil(np.log2(nbsymboles)))
    while i < len(message):
        currSymb = dictsymb[dictbin.index(message[i])]
        decodedMessage += currSymb
        # add the new symbol to the dictionary
        nbsymboles += 1
        nbBitUsed = int(np.ceil(np.log2(nbsymboles)))
        dictbin += ["{:b}".format(nbsymboles)] 
        # add 1bit if required
        if 2**nbBitUsed > len(dictsymb):
            for j in range(nbsymboles):
                dictbin[j] = "{:b}".format(j).zfill(int(np.ceil(np.log2(nbsymboles))))
        if i+1 < len(message):
            nextSymb = dictsymb[dictbin.index(message[i+offset+1])]
        
        dictsymb += [currSymb+nextSymb]
        
        i+=1
    print(dictsymb)
    return decodedMessage
        
        

In [219]:
print("Message original : ", message)
encodedMessage = encode(message)
print("Message encodé : ", encodedMessage[0])
print("Message décodé : ", decode(encodedMessage[0], encodedMessage[1], encodedMessage[2]))

Message original :  ABAABAABACABBABCDAADACABABAAABAABBABABAABAAB
Message encodé :  ['00', '001', '000', '100', '110', '0101', '0010', '0100', '0101', '0001', '0010', '0011', '0110', '01111', '01010', '01100', '00110', '00111', '01011', '00111', '00101', '10101', '00001']
['A', 'B', 'C', 'D', 'AB', 'BA', 'AAB', 'ABAAB', 'AABBA', 'BAC', 'CAB', 'ABBA', 'BAB', 'BC', 'CD', 'DAAB', 'AABDAAB', 'DAABCAB', 'CABBAB', 'BABAAB', 'AABABAAB', 'ABAABABBA', 'ABBAABAAB', 'ABAABBA', 'BAABAABABBA', 'ABAABABBAB', 'BB']
['00000', '00001', '00010', '00011', '00100', '00101', '00110', '00111', '01000', '01001', '01010', '01011', '01100', '01101', '01110', '01111', '10000', '10001', '10010', '10011', '10100', '10101', '10110', '10111', '11000', '11001', '11010']
Message décodé :  ABAABAABBACABBABCDAABDAABCABBABAABABAABABBAABAABBAABAABABBAB
